In [1]:
from netneurotools import datasets, freesurfer
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests as fdr
from nibabel.freesurfer import read_annot, read_geometry
def _decode_list(vals):
    """ List decoder
    """
    return [val.decode() if hasattr(val, 'decode') else val for val in vals]

/Users/zhangxihan/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


### Load data and atalas

In [2]:
## 1. set dirs
base_dir = '/Volumes/GoogleDrive/My Drive/Jocelyn_Cocain' # change this to your local dir
out_dir = base_dir+'/output'
net_enrich_dir = '/Volumes/GoogleDrive/My Drive/Gradient_Shift_Cellular_Basis/Milgram/data/NetworkEnrichment'

## 2. load the 18 cell-types
LakeDFC_schaefer400 = pd.read_csv(base_dir+'/schaeffer_LAKE_DFC_400_7Net_expr_mat_new_NormZscore0.3.csv',header=None,index_col=0)
LakeVIS_schaefer400 = pd.read_csv(base_dir+'/schaeffer_LAKE_VIS_400_7Net_expr_mat_new_NormZscore0.3.csv',header=0,index_col=0)

## 3. load the annotation files of Schaefer400
annot = datasets.fetch_schaefer2018(version='fsaverage', data_dir=None, url=None, resume=True, verbose=1)['400Parcels7Networks']

### Prepare cell type fractions parceled in Schaefer 400

In [3]:
## 1. make 0 to be a small none-zero value (so that in the visualization, this doesn't look the same as NaN)
LakeDFC_schaefer400.replace(0,0.0001,inplace=True)
LakeVIS_schaefer400.replace(0,0.0001,inplace=True)

## 2. rename the column names according to annot file
vn, _, names_l = read_annot(annot.lh)
names_l = _decode_list(names_l)
vn, _, names_r = read_annot(annot.rh)
names_r = _decode_list(names_r)
LakeVIS_schaefer400.columns = names_l[1:] + names_r[1:]

## 3. make name list of NaN column (parcel index and name)
# index list of NaN column
NaN_Lake_schaefer400_index = [0] + LakeDFC_schaefer400.columns[LakeDFC_schaefer400.isna().any()].tolist()
# name list of NaN column
NaN_Lake_schaefer400_name = [names_l[0]] + LakeVIS_schaefer400.columns[LakeVIS_schaefer400.isna().any()].tolist()


## 4. make name list of row names (cell-type)
cell_types_DFC = list(LakeDFC_schaefer400.index.values)
cell_types_VIS = list(LakeVIS_schaefer400.index.values)


## 5. drop NaN columns
LakeDFC_schaefer400_drop = LakeDFC_schaefer400.dropna(axis=1)
LakeVIS_schaefer400_drop = LakeVIS_schaefer400.dropna(axis=1)
# save it
file_name = '/Empirical_LakeDFC_Schaefer400_NaNDropped.csv'
file_path = out_dir + file_name
LakeDFC_schaefer400_drop.to_csv(file_path)
file_name = '/Empirical_LakeVIS_Schaefer400_NaNDropped.csv'
file_path = out_dir + file_name
LakeVIS_schaefer400_drop.to_csv(file_path)

### Spin Cell Types
Cornblath, https://markello-spatialnulls.netlify.app/spatial_nulls.html

In [168]:
# loop through the 18 cell types
for this_cell in cell_types_DFC:
    print(this_cell)
    brain = LakeDFC_schaefer400_drop.loc[this_cell].to_numpy()
    nulls = freesurfer.spin_data(brain, lhannot=annot.lh, rhannot=annot.rh,n_rotate=1000,
                                 version='fsaverage',drop=NaN_Lake_schaefer400_name,verbose=True)
    file_name = '/Null_' + this_cell + '_Schaefer400_NaNDropped.csv'
    file_path = out_dir + file_name
    pd.DataFrame(nulls).to_csv(file_path)

DFC_Ex1
                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

...

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

In [5]:
# loop through the 18 cell types
for this_cell in cell_types_VIS:
    print(this_cell)
    brain = LakeVIS_schaefer400_drop.loc[this_cell].to_numpy()
    nulls = freesurfer.spin_data(brain, lhannot=annot.lh, rhannot=annot.rh,n_rotate=1000,
                                 version='fsaverage',drop=NaN_Lake_schaefer400_name,verbose=True)
    file_name = '/Null_' + this_cell + '_Schaefer400_NaNDropped.csv'
    file_path = out_dir + file_name
    pd.DataFrame(nulls).to_csv(file_path)

VIS_Ex1
                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

...

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver

                            00Reducing vertices to parcels:     0/10Reducing vertices to parcels:     1/10Reducing vertices to parcels:     2/10Reducing vertices to parcels:     3/10Reducing vertices to parcels:     4/10Reducing vertices to parcels:     5/10Reducing vertices to parcels:     6/10Reducing vertices to parcels:     7/10Reducing vertices to parcels:     8/10Reducing vertices to parcels:     9/10Reducing vertices to parcels:    10/10Reducing vertices to parcels:    11/10Reducing vertices to parcels:    12/10Reducing vertices to parcels:    13/10Reducing vertices to parcels:    14/10Reducing vertices to parcels:    15/10Reducing vertices to parcels:    16/10Reducing vertices to parcels:    17/10Reducing vertices to parcels:    18/10Reducing vertices to parcels:    19/10Reducing vertices to parcels:    20/10Reducing vertices to parcels:    21/10Reducing vertices to parcels:    22/10Reducing vertices to parcels:    23/10Reducing vertices to parcels:    24/10Reducing vertices to

Reducing vertices to parcels:   863/10Reducing vertices to parcels:   864/10Reducing vertices to parcels:   865/10Reducing vertices to parcels:   866/10Reducing vertices to parcels:   867/10Reducing vertices to parcels:   868/10Reducing vertices to parcels:   869/10Reducing vertices to parcels:   870/10Reducing vertices to parcels:   871/10Reducing vertices to parcels:   872/10Reducing vertices to parcels:   873/10Reducing vertices to parcels:   874/10Reducing vertices to parcels:   875/10Reducing vertices to parcels:   876/10Reducing vertices to parcels:   877/10Reducing vertices to parcels:   878/10Reducing vertices to parcels:   879/10Reducing vertices to parcels:   880/10Reducing vertices to parcels:   881/10Reducing vertices to parcels:   882/10Reducing vertices to parcels:   883/10Reducing vertices to parcels:   884/10Reducing vertices to parcels:   885/10Reducing vertices to parcels:   886/10Reducing vertices to parcels:   887/10Reducing vertices to parcels:   888/10Reducing ver